In [ ]:
import json
import os.path
import zipfile

import gensim
import lucem_illud
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas
import pandas as pd
import requests
import scipy
import seaborn as sns
import sklearn.manifold
import spacy
import wordcloud
from spacy import displacy
from spacy.symbols import VERB, nsubj
from tqdm import tqdm

# This 'magic' command makes the plots work better
# in the notebook, don't use it outside a notebook
%matplotlib inline

# Week 2 - Counting Words & Phrases (Computational Linguistics)
This week, we take text corpora that we have developed, spidered, scraped, and encoded, and we find and count words,
simple and parsed phrases and explore the statistical properties of those counts (e.g., word frequency distributions). 
Initially, we model how to search corpora for keywords or phrases. Next, we examine the distributions of terms and 
phrases across a corpus, and the correlation between different words and phrase counts. In order to do this effectively,
we coarsely disambiguate words based of part-of-speech (POS) tagging, and normalize them through stemming and 
lemmatization. Next we distinguish *important* words and phrase within the corpus, and image them with Wordls! 
Then we calculate word frequencies, conditional frequencies (the frequency of word *shock* conditional on the 
presence of word *awe*), and statistically significant collocations of lengths 2 through $n$. Finally, we calculate 
and visualize Differences (Divergences and Distances) between the word frequency distributions from two corpora.

Then we shift to focus not simply on the *ideas* in a corpus, but also extracting precise *claims*. This process 
involves a sequential pipeline of classifying and structuring tokens from text, each of which generates potentially 
useful data for the content analyst. Steps in this process, which we examine in this notebook, include: 1) tagging 
words by their part of speech (POS) to reveal the linguistic role they play in the sentence (e.g., Verb, Noun, 
Adjective, etc.); 2) tagging words as named entities (NER) such as places or organizations; 3) structuring or 
"parsing" sentences into nested phrases that are local to, describe or depend on one another; and 4) extracting 
informational claims from those phrases.

We will be introducing spaCy as a package for Computational Linguistics, and also walk you through how to use 
the Davies Corpora.

While we wish to avoid using NLTK for heavy corpus analysis, some of its smaller functions can still be useful.

Of course, for final projects that draw on these tools, we encourage you to install the software on your own 
machines or shared servers at the university (RCC, SSRC) in order to perform these operations on much more text.

For this notebook we will be using the following packages:


# Retrieving exemplary corpora

To get started we will need some examples. Let's start by downloading one of the corpuses from the 
Davies set purchased for this class and developed by a computational linguist at Brigham Young University. 
We can get a list of works available from the [corpora here](https://www.english-corpora.org/).

In this notebook, I will be accessing the data locally - you also have access to this data through the 
Dropbox download link (shared on Canvas, not here, because the GitHub is publicly accessible). The same 
code can be modified slighlty to access the rest of the corpora on RCC. There is a notebook describing 
this process in the same repository, as well as in the same directory on RCC.

In [ ]:
corpus_file_path = (
    "/Users/michaelp/Documents/GitHub/computational-content-analysis/data/Movies"
)

# corpus_name = "Movies"

Just to be clear, your corpus_name should be different since you're not bhargav. Modify the corpus_file_path 
with your directory. This method extracts the text from the zip files and stores the raw data.

In [ ]:
def load_corpus(corpus_name, corpus_style="text"):
    texts_raw = {}
    for file_name in os.listdir(corpus_name + "/"):
        if corpus_style in file_name:
            print(file_name)
            z_files = zipfile.ZipFile(corpus_name + "/" + file_name)
            for z_file in z_files.namelist():
                texts_raw[z_file] = []
                with z_files.open(z_file) as f:
                    for line in f:
                        texts_raw[z_file].append(line)
    return texts_raw

In [ ]:
movie_raw = load_corpus(corpus_file_path)

In [ ]:
movie_raw["11.txt"][0:3]

It seems messy, but nothing we can't clean. This basic method replaces some of the issues with the formatting, 
and prints the errors if any for debugging. Let us clean one of the raw text files.

We ignore the non-unicode data for convenience in our first pass.

In [ ]:
def clean_raw_text(raw_texts):
    clean_texts = []
    for r_text in raw_texts:
        try:
            r_text = r_text.decode("utf-8")
            clean_text = (
                r_text.replace(" 'm", "'m")
                .replace(" 'll", "'ll")
                .replace(" 're", "'re")
                .replace(" 's", "'s")
                .replace(" 're", "'re")
                .replace(" n't", "n't")
                .replace(" 've", "'ve")
                .replace(" /'d", "'d")
                .replace(" ... ", "... ")
                .replace(" , ", ", ")
                .replace(" ! ", "! ")
                .replace(" ? ", "? ")
                .replace(" . ", ". ")
                .replace("'", "")
            )
            clean_texts.append(clean_text)
        except AttributeError:
            # print("ERROR CLEANING")
            # print(text)
            continue
        except UnicodeDecodeError:
            # print("Unicode Error, Skip")
            continue
    return clean_texts

In [ ]:
clean_11 = clean_raw_text(movie_raw["11.txt"])

In [ ]:
clean_11[1]

This method cleaned a few basic formatting errors. You are welcome to experiment and add your own code 
to expand on cleaning the text. Once it begins to look the way you want it to, we can start using spaCy 
to do some heavier cleaning. spaCy does this through the use of language models. Make sure you have the 
spaCy english language model loaded.

In [ ]:
nlp = spacy.load("en_core_web_sm")

This model is going to be the basis of most of the tasks which NLTK (the Natural Language Tool Kit) 
might have done otherwise. While NLTK uses a bunch of clunky models from multiple places, spacy neatly 
packs all functionality in one model. This deep trained model will start by doing our word tokenizing 
and normalising. These methods are already included in lucem illud, and here we will just demonstrate 
them outside the package so to explain what goes on under the hood.

When we run text through a spacy model, it performs a lot of computation on each of the tokens. Each 
token is given certain attributes, such as a part of speech, information whether it is a number, a 
stop word, and so on.

To perform a tokenization, we check if the string includes punctuation and add this token.

In [ ]:
def word_tokenize(word_list):
    tokenized = []
    # pass word list through language model.
    doc = nlp(word_list)
    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            tokenized.append(token.text)
    return tokenized

In [ ]:
word_tokenize(clean_11[1])

You can see how the sentence is broken into its constituent words. We can then do some analysis with this.

Note: we still don't know which movie this is, just that it is raw movie text data. Maybe in the process 
of our text cleaning and counting we will come across something!

# Counting words

If we want to do some analysis we can start by simply counting the number of times each word occurs within the corpus.

In [ ]:
def word_counter(word_lst):
    wordCounts = {}
    for w in word_lst:
        # We usually need to normalize the case
        w_lower = w.lower()
        if w_lower in wordCounts:
            wordCounts[w_lower] += 1
        else:
            wordCounts[w_lower] = 1
    # convert to DataFrame
    countsForFrame = {"word": [], "count": []}
    for w, c in wordCounts.items():
        countsForFrame["word"].append(w)
        countsForFrame["count"].append(c)
    return pandas.DataFrame(countsForFrame)

In [ ]:
countedWords = word_counter(word_tokenize(clean_11[1]))
countedWords[:10]

Let's sort and plot our counts to investigate the shape of our word frequency distribution.

First we need to sort the words by count.

In [ ]:
# Doing this in place (changing the original DataFrame) as we don't need the unsorted DataFrame
countedWords.sort_values("count", ascending=False, inplace=True)
countedWords[:100]

Punctuation and very common words (articles 'a' and 'the'; prepositions 'of' and 'to') make 
up the most common values, but this isn't very interesting and can actually get in the way of 
our analysis. We may remove these 'function words' by removing according to a stopword list, 
setting some frequency threshold, or using a weighting scheme (like tf.idf) to decrease their 
influence - we will look at these methods in more detail as we go through this tutorial.

In [ ]:
# Creating a figure and axis lets us do things like change the scaling or add a title
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(range(len(countedWords)), countedWords["count"])
plt.show()

This shows that likelihood of a word occurring is inversely proportional to its rank. This 
effect is called [Zipf's Law](https://en.wikipedia.org/wiki/Zipf%27s_law), and suggests that 
the most frequent word will occur approximately twice as often as the second most frequent word, 
three times as often as the third most frequent word, etc. Zipf's law is most easily observed by 
plotting the data on a log-log graph, with the axes being log (rank order) and log (frequency) 
resulting in a linear relationship.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(range(len(countedWords)), countedWords["count"])
ax.set_yscale("log")
ax.set_xscale("log")
plt.show()

(The same relationship occurs in many other rankings, such as population ranks of cities, 
corporation sizes, income rankings, etc.)

The distribution was imagined by Zipf to be driven by a principle of 'least effort' where 
speakers did not work any harder than necessary to communicate a given idea, but the basis 
for this relationship is still not well understood and conforms at least as well to a process 
of [preferential attachment](https://en.wikipedia.org/wiki/Preferential_attachment) whereby people disproportionately attend to popular words.


# Getting and working with raw text

First, we will retrieve a collection of press releases from a *GitHub API* that archived 
them, based on a number of analyses by Justin Grimmer, a political scientist whose work 
we will read in two weeks!

GitHub API requests are made to `'https://api.github.com/'` and responses are in JSON, 
similar to Tumblr's API.

We will get the information on [github.com/lintool/GrimmerSenatePressReleases](https://github.com/lintool/GrimmerSenatePressReleases) as it contains a nice set documents.

In [ ]:
r = requests.get("https://api.github.com/repos/lintool/GrimmerSenatePressReleases")
senateReleasesData = json.loads(r.text)
print(senateReleasesData.keys())
print(senateReleasesData["description"])

What we are interested in here is the `'contents_url'`

In [ ]:
print(senateReleasesData["contents_url"])

We can use this to get any or all of the files from the repo.

In [ ]:
r = requests.get(
    "https://api.github.com/repos/lintool/GrimmerSenatePressReleases/contents/raw/Whitehouse"
)
white_house_links = json.loads(r.text)
white_house_links[0]

Now we have a list of information about White House press releases. Let's look at one of them.

In [ ]:
r = requests.get(white_house_links[0]["download_url"])
white_house_release = r.text
print(white_house_release[:1000])
len(white_house_release)

Now we have a string of correctly encoded text. For analysis of its words, next we need to 
tokenize it, or split it into a sequence of tokens or word instances.

In [ ]:
wh_tokens = word_tokenize(white_house_release)
wh_tokens[10:30]

Previous iterations of the course used `nltk`s tokenizer, but as we saw before, we used spaCy's model 
(https://spacy.io/api/tokenizer). We intend to avoid using NLTK when we can - for production level code, 
it is a lot faster and cleaner to use spaCy.

That being said, `nltk` does have some useful methods for quick analysis of small corpora. We will 
explore a few here, and encourage you to figure out for yourself which other ones might be useful.

To use the list of tokens in `nltk`, and take advantage of functions like `concordance`, shown above, 
we can convert it into a `Text`.

In [ ]:
wh_text = nltk.Text(wh_tokens)

white_house_index = nltk.text.ConcordanceIndex(wh_text)
white_house_index.print_concordance("Whitehouse")

*Note* that the `Text` class is for doing rapid, exploratory analysis. It provides 
an easy interface to many of the operations we want to do, but it does not allow us 
much control over the particular operations it glosses. When you are doing a more 
complete analysis, you should be using the module specifically designed for that 
task instead of the shortcut method `Text` provides, e.g. use  [`collocations` Module](https://www.nltk.org/api/nltk.html#module-nltk.collocations) 
instead of `.collocations()`.

Now that we have gotten this loaded, let's glance at few features we will delve 
into more deeply later.

For example, we can find words that statistically tend to occur together and typically 
have a composite, idiomatic meaning irreducible to the semantics of its component words. 
We will do this later with more control over exactly how these are identified.

Or we can pick a word (or words) and find what words tend to occur around it.

In [ ]:
wh_text.common_contexts(["stem"])

We can also just count the number of times the word occurs.

In [ ]:
wh_text.count("cell")

We can also plot each time a set of words occurs.

In [ ]:
sns.reset_orig()  # Seaborn messes with this plot, disabling it
wh_text.dispersion_plot(
    ["Sen.", "stem", "cell", "federal", "Lila", "Barber", "Whitehouse"]
)
sns.set()  # Re-enabling seaborn

If we want to do an analysis of all the White House press releases we will first need to obtain them. 
By looking at the API we can see the URL we want is [https://api.github.com/repos/lintool/GrimmerSenatePressReleases/contents/raw/Whitehouse](https://api.github.com/repos/lintool/GrimmerSenatePressReleases/contents/raw/Whitehouse), so we can 
create a function to scrape the individual files.

If you want to know more about downloading from APIs, refer back to the 1st notebook.

In [ ]:
def get_github_files(target, max_files=100):
    # We are setting a max so our examples don't take too long to run
    # For converting to a DataFrame
    releases_dict = {
        "name": [],  # The name of the file
        "text": [],  # The text of the file, watch out for binary files
        "path": [],  # The path in the git repo to the file
        "html_url": [],  # The url to see the file on GitHub
        "download_url": [],  # The url to download the file
    }

    # Get the directory information from GitHub
    r = requests.get(target)
    filesLst = json.loads(r.text)

    for file_dict in filesLst[:max_files]:
        # These are provided by the directory
        releases_dict["name"].append(file_dict["name"])
        releases_dict["path"].append(file_dict["path"])
        releases_dict["html_url"].append(file_dict["html_url"])
        releases_dict["download_url"].append(file_dict["download_url"])

        # We need to download the text though
        text = requests.get(file_dict["download_url"]).text
        releases_dict["text"].append(text)

    return pandas.DataFrame(releases_dict)


wh_releases = get_github_files(
    "https://api.github.com/repos/lintool/GrimmerSenatePressReleases/contents/raw/Whitehouse",
    max_files=10,
)
wh_releases[:5]

If you are having trouble downloading the data uncomment this next cell

In [ ]:
# wh_releases = pandas.read_csv('../data/whReleases.csv')

Now we have all the texts in a DataFrame we can look at a few things.

First let's tokenize the texts with the same tokenizer as we used before. We will just 
save the tokens as a list for now; no need to convert to `Text`s.

In [ ]:
wh_releases["tokenized_text"] = wh_releases["text"].apply(lambda x: word_tokenize(x))

Now lets see how long each of the press releases is:

In [ ]:
wh_releases["word_counts"] = wh_releases["tokenized_text"].apply(lambda x: len(x))
wh_releases["word_counts"]

## <font color="red">*Exercise 1*</font>

<font color="red">Construct cells immediately below this that input and interrogate a corpus 
relating to your anticipated final project. This could include one of the Davies corpora or 
some other you have constructed. Turn your text into a nltk `Text` object, and explore all 
the features examined above, and others that relate to better understanding your corpus in 
relation to your research question. </font>

# Filtering and normalizing text

As we want to start comparing the different releases we may choose to do a bit of filtering and normalizing that will allow us to focus on what we most care about. We can first make all of the words lower case, then drop the non-word tokens. Next, we can remove some 'stop words', stem the remaining words to remove suffixes, prefixes and (in some languages) infixes, or lemmatize tokens by intelligently grouping inflected or variant forms of the same word (e.g., with a stemmer and a dictionary).

To begin this process, we will first define a function to work over the tokenized lists, then another to add normalized tokens to a new column.

We will be using spaCy's built in stop words identifying capacity. When we run text through spaCy's language pipeline, it automatically tags it as a stop word.

We can remove these stopwords from the analysis by fiat, but we could also take a more principled approach by looking at the frequency distribution of words and selecting a specific cut-off associated with the preservation of 'meaningful words' identified upon inspection. Alternatively, we could automatically set a cut-off by rule, such as removal of all words more frequent then the most frequent verb, or the most frequent noun (not pronoun), or some term of central interest.

In [ ]:
countsDict = {}
for word in wh_releases["tokenized_text"].sum():
    if word in countsDict:
        countsDict[word] += 1
    else:
        countsDict[word] = 1
word_counts = sorted(countsDict.items(), key=lambda x: x[1], reverse=True)
word_counts[:20]

We can look at this and pick the cutoff, often at the first noun. So we will cut all words before `'Whitehouse'`.

In [ ]:
# The stop list is then all words that occur before the first noun
stop_words_freq = []
for word, count in word_counts:
    if word == "Whitehouse":
        break
    else:
        stop_words_freq.append(word)
stop_words_freq

If we did have any extra words to add as stop words apart from those normally considered as stop words, we would add it to our stop words by passing it to the normalize tokens method.

Now we can create our function to normalize the tokens

In [ ]:
def normalize_tokens(word_list, extra_stop=None):
    # We can use a generator here as we just need to iterate over it
    if extra_stop is None:
        extra_stop = []
    normalized = []
    if type(word_list) == list and len(word_list) == 1:
        word_list = word_list[0]

    if type(word_list) == list:
        word_list = " ".join([str(elem) for elem in word_list])

    doc = nlp(word_list.lower())

    # add the property of stop word to words considered as stop words
    if len(extra_stop) > 0:
        for stopword in extra_stop:
            lexeme = nlp.vocab[stopword]
            lexeme.is_stop = True

    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if (
            w.text != "\n"
            and not w.is_stop
            and not w.is_punct
            and not w.like_num
            and len(w.text.strip()) > 0
        ):
            # we add the lemmatized version of the word
            normalized.append(str(w.lemma_))

    return normalized

In [ ]:
wh_releases["normalized_tokens"] = wh_releases["tokenized_text"].apply(
    lambda x: normalize_tokens(x)
)

wh_releases["normalized_tokens_count"] = wh_releases["normalized_tokens"].apply(
    lambda x: len(x)
)

wh_releases

You can see that in the final step of normalizing, we add the lemmatized form of the word. spaCy's built in lemmatizer does this (https://spacy.io/api/lemmatizer). Lemmatization can be thought of as finding the root or the base of the word after removing inflections and other variations of words. Another possible way of finding the roots of words is using NLTK's porting or stemming functionalities, but we will not be getting into them.

Now that it is cleaned we start analyzing the dataset. We can start by finding frequency distributions for the dataset. Lets start looking at all the press releases together. The [`ConditionalFreqDist`](http://www.nltk.org/api/nltk.html#nltk.probability.ConditionalProbDist) class reads in an iterable of tuples, the first element is the condition and the second the focal word. For starters, we will use word lengths as the conditions, but tags or clusters will provide more useful results.

In [ ]:
# .sum() adds together the lists from each row into a single list
whcf_dist = nltk.ConditionalFreqDist(
    ((len(w), w) for w in wh_releases["normalized_tokens"].sum())
)

# print the number of words
print(whcf_dist.N())

From this we can lookup the distributions of different word lengths

In [ ]:
whcf_dist[3].plot()

We can also create a Conditional Probability Distribution or [`ConditionalProbDist`](http://www.nltk.org/api/nltk.html#nltk.probability.ConditionalProbDist) from the `ConditionalFreqDist`. To do this, however, we need a model for the probability distribution. A simple model is [`ELEProbDist`](http://www.nltk.org/api/nltk.html#nltk.probability.ELEProbDist) which gives the expected likelihood estimate for the probability distribution of the experiment used to generate the observed frequency distribution.

In [ ]:
whcf_dist = nltk.ConditionalProbDist(whcf_dist, nltk.ELEProbDist)

# print the most common 2-letter word
print(whcf_dist[2].max())

# And its probability
print(whcf_dist[2].prob(whcf_dist[2].max()))

Length is one feature of a word, but there are many more important features we care about. Another 
critical feature is a word's role in the sentence, or its part of speech (POS).

The method below works similarly to ```nltk.pos_text```, and can work with the conditional 
frequency distributions it provides.

In [ ]:
def spacy_pos(word_list):
    tags = []
    doc = nlp(word_list.lower())
    for w in doc:
        tags.append((w.text, w.tag_))
    return tags

In [ ]:
wh_releases["normalized_tokens_POS"] = [spacy_pos(t) for t in wh_releases["text"]]

This gives us a new column with the part of speech as a short initialism and the word in a 
tuple, exactly how the `nltk.ConditionalFreqDist()` function wants them. We can now 
construct another conditional frequency distribution.

In [ ]:
whcfdist_WordtoPOS = nltk.ConditionalFreqDist(
    wh_releases["normalized_tokens_POS"].sum()
)
list(whcfdist_WordtoPOS.items())[:10]

This gives the frequency of each word as each part of speech...which can 
be uninformative and boring. What we want is the converse; the frequency 
of each part of speech for each word.

In [ ]:
whcfdist_POStoWord = nltk.ConditionalFreqDist(
    (p, w) for w, p in wh_releases["normalized_tokens_POS"].sum()
)

We can now identify and collect all of the superlative adjectives

In [ ]:
whcfdist_POStoWord["JJ"]

Or look at the most common nouns

In [ ]:
whcfdist_POStoWord["NN"].most_common(5)

Or plot the base form verbs against their number of occurrences

In [ ]:
whcfdist_POStoWord["VB"].plot()

We can then do a similar analysis of the word probabilities

In [ ]:
whcpdist_POStoWord = nltk.ConditionalProbDist(whcfdist_POStoWord, nltk.ELEProbDist)

# print the most common nouns
print(whcpdist_POStoWord["NN"].max())

# And its probability
print(whcpdist_POStoWord["NN"].prob(whcpdist_POStoWord["NN"].max()))

We can even create a...wait for it...*WORD CLOUD* or Wordl to gaze at and draw 
mystical, approximate inferences about important nouns and verbs in our corpus:

In [ ]:
wc = wordcloud.WordCloud(
    background_color="white",
    max_words=500,
    width=1000,
    height=1000,
    mode="RGBA",
    scale=0.5,
).generate(" ".join(wh_releases["normalized_tokens"].sum()))
plt.imshow(wc)
plt.axis("off")
plt.savefig("whitehouse_word_cloud.pdf", format="pdf")

## <font color="red">*Exercise 2*</font>

<font color="red">Construct cells immediately below this that filter, stem and lemmatize the tokens in your corpus, and then creates plots (with titles and labels) that map the word frequency distribution, word probability distribution, and at least two conditional probability distributions that help us better understand the social and cultural game underlying the production of your corpus. Create a wordl of words (or normalized words) and add a few vague comments about what mysteries are revealed through it.</font>

## Davies Corpora

Now that we have our basic cleaning down, we can arrange our Davies Corpora. Let 
us try this with the movies corpora we have already loaded.

We'll use a smaller corpus of a 1000 movies for this demonstration.

In [ ]:
zfile = zipfile.ZipFile(corpus_file_path + "/sources_movies.zip")
source = []

In [ ]:
for file in zfile.namelist():
    with zfile.open(file) as f:
        for line in f:
            source.append(line)

In [ ]:
source[0:20]

It looks dirty because the file is encoded as bytes, but we can certainly see 
the information there. The file id is also present in the original raw text 
data: as the first "word". Look back at the normalized/tokenized words to 
confirm that. We're going to use this to create a dataframe with: Fileid, 
movie name, genre, year, and country.

It is advised that you run a similar check of the source file before you 
do other extraction.

First, let us create a dictionary mapping file-id to all the text. Each 
movie will be mapped to a list of the tokenized words.

In this example, I only use it to load 100 movies. You can comment this 
out or increase/decrease the number as you see fit.

Note that in the following cells, we make use of the `lucem_illud` package's 
`word_tokenize` function. You can see that function on line 70 of [processing.py](https://github.com/UChicago-Computational-Content-Analysis/lucem_illud/blob/main/lucem_illud/processing.py), and we provide 
the code here for transparency. Since we put this folder in our helper package, 
you won't need to define this function every time you want to use it; you
just `import lucem_illud`!

In [ ]:
# processing.py calls a spacy model called "en" (and sometimes called "en_core_web_sm") so we load that first.
try:
    nlp = spacy.load("en")
except OSError:
    nlp = spacy.load("en_core_web_sm")


# Here is the function from lucem_illud/lucem_illud/processing.py
def word_tokenize(word_list, model=nlp, MAX_LEN=1500000):
    tokenized = []
    if type(word_list) == list and len(word_list) == 1:
        word_list = word_list[0]

    if type(word_list) == list:
        word_list = " ".join([str(elem) for elem in word_list])
    # since we're only tokenizing, I remove RAM intensive operations and increase max text size

    model.max_length = MAX_LEN
    doc = model(word_list, disable=["parser", "tagger", "ner"])

    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            tokenized.append(token.text)
    return tokenized

In [ ]:
movie_texts = {}

In [ ]:
for files in movie_raw:
    if len(movie_texts) > 100:
        break
    movies = clean_raw_text(movie_raw[files][1:])
    for movie in movies:
        # We can run the following line of code with or without `lucem_illud.`, using the package or our code 2 cells above.
        txts = lucem_illud.word_tokenize(movie)
        try:
            movie_texts[txts[0][2:]] = txts[1:]
        except IndexError:
            continue

In [ ]:
movie_df = pd.DataFrame(
    columns=["Movie Name", "Genre", "Year", "Country", "Tokenized Texts"]
)

In [ ]:
for movie in source[3:]:
    try:
        (
            tid,
            fileid,
            total_words,
            genre,
            year,
            lang,
            country,
            imdb,
            title,
        ) = movie.decode("utf-8").split("\t")
    except UnicodeDecodeError:
        continue
    try:
        movie_df.loc[fileid.strip()] = [
            title.strip(),
            genre.strip(),
            year.strip(),
            country.strip(),
            movie_texts[fileid.strip()],
        ]
    except KeyError:
        continue

In [ ]:
movie_df.head()

# Part-of-Speech (POS) tagging

In POS tagging, we classify each word by its semantic role in a sentence. We will be using the in-built spaCy pos-tagging.

https://spacy.io/usage/linguistic-features

|#. |Tag |Description |
|---|----|------------|
|1.	|CC	|Coordinating conjunction
|2.	|CD	|Cardinal number
|3.	|DT	|Determiner
|4.	|EX	|Existential there
|5.	|FW	|Foreign word
|6.	|IN	|Preposition or subordinating conjunction
|7.	|JJ	|Adjective
|8.	|JJR|	Adjective, comparative
|9.	|JJS|	Adjective, superlative
|10.|	LS	|List item marker
|11.|	MD	|Modal
|12.|	NN	|Noun, singular or mass
|13.|	NNS	|Noun, plural
|14.|	NNP	|Proper noun, singular
|15.|	NNPS|	Proper noun, plural
|16.|	PDT	|Predeterminer
|17.|	POS	|Possessive ending
|18.|	PRP	|Personal pronoun
|19.|	PRP\$|	Possessive pronoun
|20.|	RB	|Adverb
|21.|	RBR	|Adverb, comparative
|22.|	RBS	|Adverb, superlative
|23.|	RP	|Particle
|24.|	SYM	|Symbol
|25.|	TO	|to
|26.|	UH	|Interjection
|27.|	VB	|Verb, base form
|28.|	VBD	|Verb, past tense
|29.|	VBG	|Verb, gerund or present participle
|30.|	VBN	|Verb, past participle
|31.|	VBP	|Verb, non-3rd person singular present
|32.|	VBZ	|Verb, 3rd person singular present
|33.|	WDT	|Wh-determiner
|34.|	WP	|Wh-pronoun
|35.|	WP$	|Possessive wh-pronoun
|36.|	WRB	|Wh-adverb

spaCy pos-tags word as we run it through the english language model. A small example:

In [ ]:
sent = nlp("Tom went to IKEA to get some of those delicious Swedish meatballs.")

In [ ]:
for token in sent:
    print(token.text, token.pos_, token.tag_)

This looks quite good. Now we will try POS tagging with a somewhat larger corpus. We consider a few of the top posts from the reddit data we used last week.

In [ ]:
redditDF = pandas.read_csv("../data/reddit.csv")

Grabbing the 10 highest scoring posts and tokenizing the sentences. Once again, notice that we aren't going to do any kind of stemming this week (although *semantic* normalization may be performed where we translate synonyms into the same focal word). Here, we will use sentences tokenized by spaCy. Like before, these methods can also be found in `lucem_illud`.



In [ ]:
def sent_tokenize(word_list):
    doc = nlp(word_list)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

In [ ]:
redditTopScores = redditDF.sort_values("score")[-10:]
redditTopScores["sentences"] = redditTopScores["text"].apply(
    lambda x: [word_tokenize(s) for s in sent_tokenize(x)]
)
redditTopScores.index = range(
    len(redditTopScores) - 1, -1, -1
)  # Reindex to make things nice in the future
redditTopScores

In [ ]:
def tag_sents_pos(sentences):
    """
    function which replicates NLTK pos tagging on sentences.
    """
    new_sents = []
    for sentence in sentences:
        new_sent = " ".join(sentence)
        new_sents.append(new_sent)
    final_string = " ".join(new_sents)
    doc = nlp(final_string)

    pos_sents = []
    for sent in doc.sents:
        pos_sent = []
        for token in sent:
            pos_sent.append((token.text, token.tag_))
        pos_sents.append(pos_sent)

    return pos_sents

In [ ]:
redditTopScores["POS_sents"] = redditTopScores["sentences"].apply(
    lambda x: tag_sents_pos(x)
)

In [ ]:
redditTopScores["POS_sents"]

And count the number of `NN` (nouns)

In [ ]:
countTarget = "NN"
targetCounts = {}
for entry in redditTopScores["POS_sents"]:
    for sentence in entry:
        for ent, kind in sentence:
            if kind != countTarget:
                continue
            elif ent in targetCounts:
                targetCounts[ent] += 1
            else:
                targetCounts[ent] = 1
sortedTargets = sorted(targetCounts.items(), key=lambda x: x[1], reverse=True)
sortedTargets[:20]

What about the adjectives that modify the word, "computer"?

In [ ]:
NTarget = "JJ"
Word = "computer"
NResults = set()
for entry in redditTopScores["POS_sents"]:
    for sentence in entry:
        for (ent1, kind1), (ent2, kind2) in zip(sentence[:-1], sentence[1:]):
            if (kind1, ent2.lower()) == (NTarget, Word):
                NResults.add(ent1)
            else:
                continue

print(NResults)

## <font color="red">Exercise 3</font>

<font color="red">In the cells immediately following, perform POS tagging on a meaningful (but modest) 
subset of a corpus associated with your final project. Examine the list of words associated with at 
least three different parts of speech. Consider conditional associations (e.g., adjectives associated 
with nouns or adverbs with verbs of interest). What do these distributions suggest about your corpus?

# Collocations

As illustrated above, we might want to find significant bigrams (two-word phrases), trigrams (three-word phrases), n-grams (*n*-word phrases) or skip-grams (noncontinguous 'phrases' with skip-length *n*).

We will begin with the [`nltk.collocations.BigramCollocationFinder`](http://www.nltk.org/api/nltk.html?highlight=bigramcollocationfinder#nltk.collocations.BigramCollocationFinder) class, which can be given raw lists of strings with the `from_words()` method. By default it only looks at continuous bigrams but there is an option (`window_size`) to allow skip-grams.

In [ ]:
whBigrams = nltk.collocations.BigramCollocationFinder.from_words(
    wh_releases["normalized_tokens"].sum()
)
print("There are {} bigrams in the finder".format(whBigrams.N))

To compare the bigrams we need to tell nltk what our score function is. Initially, we will look at the raw counts.

In [ ]:
def bigramScoring(count, wordsTuple, total):
    return count


print(whBigrams.nbest(bigramScoring, 10))

One note about how `BigramCollocationFinder` works. It doesn't use the strings internally.

In [ ]:
birgramScores = []


def bigramPrinting(count, wordsTuple, total):
    global birgramScores
    birgramScores.append(
        "The first word is:  {}, The second word is: {}".format(*wordsTuple)
    )
    # Returns None so all the tuples are considered to have the same rank


whBigrams.nbest(bigramPrinting, 10)
print("\n".join(birgramScores[:10]))

The words are each given numeric IDs and there is a dictionary that maps the IDs to 
the words they represent. This is a common performance optimization.

Two words can appear together by chance. Recall from  Manning and Schütze's textbook 
that a t-value can be computed for each bigram to see how significant the association 
is. You may also want to try computing the $\chi^2$, likelihood ratio, and point-wise 
mutual information statistics.

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
whBigrams.score_ngrams(bigram_measures.likelihood_ratio)[:40]
# other options include student_t, chi_sq, likelihood_ratio, pmi

There are a few other available measures

In [ ]:
[s for s in dir(bigram_measures) if s[0] != "_"]

We can also look at trigrams

In [ ]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
whTrigrams = nltk.collocations.TrigramCollocationFinder.from_words(
    wh_releases["normalized_tokens"].sum()
)
whTrigrams.score_ngrams(trigram_measures.student_t)[:20]

Or n-grams (for any number n)

In [ ]:
Ngrams = nltk.ngrams(wh_releases["normalized_tokens"].sum(), 4)
Counts = {}
for ngram in list(Ngrams):
    if ngram in Counts.keys():
        Counts[ngram] += 1
    else:
        Counts[ngram] = 1
Filtered = {}
for key in Counts.keys():
    if Counts[key] < 2:
        pass
    else:
        Filtered[key] = Counts[key]
print(Filtered)

Gensim is another important python text processing package which allows us to use 
collocations, among other cool methods (which we will explore later weeks).

In [ ]:
bigram = gensim.models.Phrases(wh_releases["normalized_tokens"])

In [ ]:
texts = [bigram[line] for line in wh_releases["normalized_tokens"]]

In [ ]:
texts[0]

## <font color="red">*Exercise 4*</font>

<font color="red">Construct cells immediately below this that identify statistically 
significant bigrams, trigrams, quadgrams, higher-order *n*grams and skipgrams. Explore 
whether these collocations are idiomatic and so irreducible to the semantic sum of their 
component words. You can do this by examination of conditional frequencies (e.g., what 
else is 'united' besides the 'United States'). If these phrases are idiomatic, what do 
they suggest about the culture of the world producing them?

<font color="red">**Stretch**: In Manning and Schütze's textbook, there Section 5.3.2 
explores how to use the *t*-test to find words whose co-occurance patterns best 
distinguish two words. Implement that and use it to explore phrases in your corpus. 
For instance, you could tell what words come after "America" much more often than 
after "Iraq"?</font>

# Named-Entity Recognition

Named Entity Recognition (NER) is also a classification task, which identifies 
named objects.
Like every other part of the pipeline, spaCy also tags words with their tagged 
entity (https://spacy.io/api/entityrecognizer). You can see the full capacity 
of what spaCy does by checking out: https://spacy.io/usage/linguistic-features.

For training the identification of such entities, spaCy uses a deep learning model.

Let's try it on a basic sentence:

In [ ]:
sent = nlp("Tom went to IKEA to get some of those delicious Swedish meatballs.")

In [ ]:
for token in sent:
    print(token.text, token.ent_type_)

For only finding ents-

In [ ]:
for ent in sent.ents:
    print(ent.text, ent.label_)

In [ ]:
def tag_sents_ner(sentences):
    """
    function which replicates NLTK ner tagging on sentences.
    """
    new_sents = []
    for sentence in sentences:
        new_sent = " ".join(sentence)
        new_sents.append(new_sent)
    final_string = " ".join(new_sents)
    doc = nlp(final_string)

    pos_sents = []
    for sent in doc.sents:
        pos_sent = []
        for ent in sent.ents:
            pos_sent.append((ent.text, ent.label_))
        pos_sents.append(pos_sent)

    return pos_sents

We can also run NER over our entire corpus:

In [ ]:
redditTopScores["classified_sents"] = redditTopScores["sentences"].apply(
    lambda x: tag_sents_ner(x)
)

In [ ]:
redditTopScores["classified_sents"]

Find the most common entities (which are, of course, boring):

In [ ]:
entityCounts = {}
for entry in redditTopScores["classified_sents"]:
    for sentence in entry:
        for ent, kind in sentence:
            if ent in entityCounts:
                entityCounts[ent] += 1
            else:
                entityCounts[ent] = 1
sortedEntities = sorted(entityCounts.items(), key=lambda x: x[1], reverse=True)
sortedEntities[:10]

Or those occurring only twice:

In [ ]:
[x[0] for x in sortedEntities if x[1] == 2]

We could also list the most common "non-objects". (We note that we're not graphing these because there are so few here.)

In [ ]:
nonObjCounts = {}
for entry in redditTopScores["classified_sents"]:
    for sentence in entry:
        for ent, kind in sentence:
            if kind == "ORG":
                continue
            elif ent in nonObjCounts:
                nonObjCounts[ent] += 1
            else:
                nonObjCounts[ent] = 1
sortedNonObj = sorted(nonObjCounts.items(), key=lambda x: x[1], reverse=True)
sortedNonObj[:10]

What about the Organizations?

In [ ]:
OrgCounts = {}
for entry in redditTopScores["classified_sents"]:
    for sentence in entry:
        for ent, kind in sentence:
            if kind == "ORG":
                print(ent)

These, of course, have much smaller counts.

In this particular example we didn't find the most interesting entities - you can remedy that in exercise 4!

## <font color="red">*Exercise 5*</font>

<font color="red">In the cells immediately following, perform NER on a (modest) 
subset of your corpus of interest. List all of the different kinds of entities 
tagged? What does their distribution suggest about the focus of your corpus? For 
a subset of your corpus, tally at least one type of named entity and calculate 
the Precision, Recall and F-score for the NER classification just performed.

## Dependency parsing and graph representations

Dependency parsing was developed to robustly capture linguistic dependencies from text. 
The complex tags associated with these parses are detailed [here]('http://universaldependencies.org/u/overview/syntax.html'). When parsing with 
the dependency parser, we will work directly from the untokenized text. Note that no 
*processing* takes place before parsing sentences--we do not remove so-called stop 
words or anything that plays a syntactic role in the sentence, although anaphora 
resolution and related normalization may be performed before or after parsing to 
enhance the value of information extraction.

We will be using spaCy's built in dependancy parser to go about exploring the tree.
Note that unlike previous examples, large corpuses or collections of sentences don't 
give us as much information, so we will be navigating smaller sentences.

Let us start by exploring noun chunks. Noun chunks are “base noun phrases” – flat 
phrases that have a noun as their head. You can think of noun chunks as a noun plus 
the words describing the noun – for example, “the lavish green grass” or “the world’s 
largest tech fund”. To get the noun chunks in a document, simply iterate over the 
sentences noun chunks.

In [ ]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")

In [ ]:
displacy.render(doc, style="dep")

In [ ]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

spaCy uses the terms head and child to describe the words connected by a single arc 
in the dependency tree. The term dep is used for the arc label, which describes the 
type of syntactic relation that connects the child to the head. As with other 
attributes, the value of .dep is a hash value. You can get the string value with .dep_.

For your reference:

Text: The original token text.

Dep: The syntactic relation connecting child to head.

Head text: The original text of the token head.

Head POS: The part-of-speech tag of the token head.

Children: The immediate syntactic dependents of the token.

In [ ]:
for token in doc:
    print(
        token.text,
        token.dep_,
        token.head.text,
        token.head.pos_,
        [child for child in token.children],
    )

Because the syntactic relations form a tree, every word has exactly one head. You 
can therefore iterate over the arcs in the tree by iterating over the words in the 
sentence. This is usually the best way to match an arc of interest — from below:

In [ ]:
verbs = set()
for possible_subject in doc:
    if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)
print(verbs)

You can *also* navigate from above. It isn't as efficient though.

In [ ]:
verbs = []
for possible_verb in doc:
    if possible_verb.pos == VERB:
        for possible_subject in possible_verb.children:
            if possible_subject.dep == nsubj:
                verbs.append(possible_verb)
                break

In [ ]:
print(verbs)

### Iterating around the local tree
A few more convenience attributes are provided for iterating around the local tree from the token. Token.lefts and Token.rights attributes provide sequences of syntactic children that occur before and after the token. Both sequences are in sentence order. There are also two integer-typed attributes, Token.n_lefts and Token.n_rights that give the number of left and right children.

In [ ]:
doc = nlp("bright red apples on the tree")
print([token.text for token in doc[2].lefts])  # ['bright', 'red']
print([token.text for token in doc[2].rights])  # ['on']
print(doc[2].n_lefts)  # 2
print(doc[2].n_rights)  # 1

You can get a whole phrase by its syntactic head using the Token.subtree attribute. This returns an ordered sequence of tokens. You can walk up the tree with the Token.ancestors attribute, and check dominance with Token.is_ancestor

In [ ]:
doc = nlp("Credit and mortgage account holders must submit their requests")

root = [token for token in doc if token.head == token][0]
subject = list(root.lefts)[0]
for descendant in subject.subtree:
    assert subject is descendant or subject.is_ancestor(descendant)
    print(
        descendant.text,
        descendant.dep_,
        descendant.n_lefts,
        descendant.n_rights,
        [ancestor.text for ancestor in descendant.ancestors],
    )

Finally, the .left_edge and .right_edge attributes can be especially useful, 
because they give you the first and last token of the subtree. This is the 
easiest way to create a Span object for a syntactic phrase. Note that 
.right_edge gives a token within the subtree — so if you use it as the 
end-point of a range, don’t forget to +1!

In [ ]:
span = doc[doc[4].left_edge.i : doc[4].right_edge.i + 1]
with doc.retokenize() as retokenizer:
    retokenizer.merge(span)
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

In [ ]:
displacy.render(doc, style="dep")

### Information Extraction

How can dependancy graphs be useful? We can extract information from the sentences based on relationships between words within parsed phrases. Let us see if we can do this to glean any information, from, say, our movies dataset.

How violent are the movies? Who is killing who?

In [ ]:
movie_violences = {}

In [ ]:
tqdm.pandas()

for index, row in movie_df[0:100].iterrows():
    text = " ".join(row["Tokenized Texts"])
    movie_violences[row["Movie Name"]] = ([], [])
    doc = nlp(text)
    for chunk in doc.noun_chunks:
        subject = 0
        object_ = 0
        # if the verb or the root of the sentence is kill
        if chunk.root.head.text == "kill":
            # we find the subjects and objects around the word, and if it does exist, add it to the tuple
            if chunk.root.dep_ == "nsubj":
                subject = chunk.root.text
            if chunk.root.dep_ == "dobj":
                object_ = chunk.root.text
            if subject != 0:
                movie_violences[row["Movie Name"]][0].append(subject)
            if object_ != 0:
                movie_violences[row["Movie Name"]][1].append(object_)

In [ ]:
movie_violences

That's pretty neat: we can see who tried to kill who, or at least talked about 
it. A lot of 'I' killing 'you', as we'd imagine. What else can we see?

The movie, about time, which is about time travel, funnily enough has:
`'About Time': (['I'], ['Hitler'])`

## <font color="red">*Exercise 6*</font>

<font color="red">In the cells immediately following, parse a (modest) subset of your 
corpus of interest. How deep are the phrase structure and dependency parse trees nested? 
How does parse depth relate to perceived sentence complexity? What are five things you 
can extract from these parses for subsequent analysis? (e.g., nouns collocated in a 
noun phrase; adjectives that modify a noun; etc.) Capture these sets of things for 
a focal set of words (e.g., "Bush", "Obama", "Trump"). What do they reveal about 
the roles that these entities are perceived to play in the social world inscribed 
by your texts?

# Distributional distances

If we want to compare different corpora, we need a distance or divergence that compares the two distributions.

We will use the:

+ Kullback-Leibler (KL) divergence
+ $\chi^2$ divergence
+ Kolmogorov-Smirnov (KS) distance
+ Wasserstein distance

### Kullback-Leibler and $x^2$ divergences ###

KL and $\chi^2$ divergences are members of the broader <a "href=https://en.wikipedia.org/wiki/F-divergence" target="_blank">$f$-divergence</a> family, a function of $D_f (P || Q)$ that calculates the difference between two probability distributions P and Q. The KL $f(t)$ is $ t \text{ log } t $, while the $\chi^2$ is $t^2-1$. KL comes from information and $\chi^2$ from measure theory. As such, the KL divergence computes the relative entropy between two distributions--how they differ in bits, while the $\chi^2$ whether the same statistical inferences can be drawn from them both.  

Specifically, given two discrete probability distributions $P$ and $Q$, the Kullback-Leibler divergence from $Q$ to $P$ is defined as:

$D_{\mathrm{KL}}(P\|Q) = \sum_i P(i) \, \log\frac{P(i)}{Q(i)}$.

The [scipy.stats.entropy()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html) function does the calculation for you, which takes in two arrays of probabilities and computes the KL divergence. Note that the KL divergence is in general not commutative, i.e. $D_{\mathrm{KL}}(P\|Q) \neq D_{\mathrm{KL}}(Q\|P)$ .

Also note that the KL divernce is the sum of elementwise divergences. Scipy provides [scipy.special.kl_div()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.kl_div.html#scipy-special-kl-div) which calculates elementwise divergences for you.

The $\chi^2$ Divergence is defined as:

$D_{\mathrm{\chi^2}}(P\|Q) = \sum_i \left(\frac{P(i)}{Q(i)}-1\right)^2$.

This is also noncommutative, and the code can be drawn directly from scipy.

### Kolmogorov-Smirnov ###

The two-sample Kolmogovorov-Smirnov test statistic calculates the distance between the cumulative distribution function of the two distributions to be compared, and, along with the $x^2$ divergence, is among the most common approaches two calculating a distance in statistics. It can be interpreted as a test of whether two distributions are drawn from the same underlying distribution. As with the others, the code is readily available in scipy.

### Wasserstein Distance ###

When this is computed on a Euclidian metric structure (e.g., numbers of words), this is also known as the earth mover’s distance, because it can be seen as the minimum amount of "work" required to transform $P$ into $Q$, where "work" is measured as the amount of distribution weight that must be moved, multiplied by the distance it has to be moved.

### Computing ###

To do this we will need to create the arrays, lets compare the Whitehouse releases with the Kennedy releases. First we have to download them and load them into a DataFrame.

In [ ]:
kenReleases = get_github_files(
    "https://api.github.com/repos/lintool/GrimmerSenatePressReleases/contents/raw/Kennedy",
    max_files=10,
)
kenReleases[:5]

Then we can tokenize, stem and remove stop words, like we did for the Whitehouse press releases

In [ ]:
kenReleases["tokenized_text"] = kenReleases["text"].apply(lambda x: word_tokenize(x))
kenReleases["normalized_tokens"] = kenReleases["tokenized_text"].apply(
    lambda x: normalize_tokens(x)
)

Now we need to compare the two collection of words, remove those not found in both, and assign the remaining ones indices.

In [ ]:
whWords = set(wh_releases["normalized_tokens"].sum())
kenWords = set(kenReleases["normalized_tokens"].sum())

# Change & to | if you want to keep all words
overlapWords = whWords & kenWords

overlapWordsDict = {word: index for index, word in enumerate(overlapWords)}
overlapWordsDict["student"]

Now we can count the occurrences of each word in the corpora and create our arrays. Note that we don't have to use numpy arrays as we do here. We could just use a list, but the arrays are faster in numpy so we encourage you to get in the habit of using them.

In [ ]:
def makeProbsArray(dfColumn, overlapDict):
    words = dfColumn.sum()
    countList = [0] * len(overlapDict)
    for word in words:
        try:
            countList[overlapDict[word]] += 1
        except KeyError:
            # The word is not common so we skip it
            pass
    countArray = np.array(countList)
    return countArray / countArray.sum()


whProbArray = makeProbsArray(wh_releases["normalized_tokens"], overlapWordsDict)
kenProbArray = makeProbsArray(kenReleases["normalized_tokens"], overlapWordsDict)
kenProbArray.sum()
# There is a little bit of a floating point math error
# but it's too small to see with print and too small matter here

We can now compute the KL divergence. Pay attention to the asymmetry. Use [the Jensen–Shannon divergence](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence), which is the average KL divergence between each distribution and the average of both distributions (i.e., the midpoint), if you want symmetry.

In [ ]:
wh_kenDivergence = scipy.stats.entropy(whProbArray, kenProbArray)
print(wh_kenDivergence)
ken_whDivergence = scipy.stats.entropy(kenProbArray, whProbArray)
print(ken_whDivergence)

Then, we can do the elementwise calculation and see which words best distinguish the two corpora.

In [ ]:
wh_kenDivergence_ew = scipy.special.kl_div(whProbArray, kenProbArray)
kl_df = pandas.DataFrame(
    list(overlapWordsDict.keys()),
    columns=["word"],
    index=list(overlapWordsDict.values()),
)
kl_df = kl_df.sort_index()
kl_df["elementwise divergence"] = wh_kenDivergence_ew
kl_df[:10]

In [ ]:
kl_df.sort_values(by="elementwise divergence", ascending=False)[:10]

Now we will apply what we learned today
First, let's transform every text into normalized tokens. Note that in this first step, no stopword is removed.
We'll use only the first 11 movies: you are welcome to try more.

In [ ]:
corpora = []
for index, row in movie_df.iterrows():
    if len(corpora) > 10:
        break
    corpora.append(row["Tokenized Texts"])

Then, let's separate the normalized tokens into stopwords and non-stopwords.

In [ ]:
corpora_s = []
corpora_nons = []
for corpus in corpora:
    s = []
    nons = []
    doc = nlp(" ".join(corpus))
    for word in doc:
        if word.is_stop:
            s.append(word.text)
        else:
            nons.append(word.text)
    corpora_s.append(s)
    corpora_nons.append(nons)

Define some covenient funtions for calculating divergence and distances.

In [ ]:
def kl_divergence(X, Y):
    P = X.copy()
    Q = Y.copy()
    P.columns = ["P"]
    Q.columns = ["Q"]
    df = Q.join(P).fillna(0)
    p = df.iloc[:, 1]
    q = df.iloc[:, 0]
    D_kl = scipy.stats.entropy(p, q)
    return D_kl


def chi2_divergence(X, Y):
    P = X.copy()
    Q = Y.copy()
    P.columns = ["P"]
    Q.columns = ["Q"]
    df = Q.join(P).fillna(0)
    p = df.iloc[:, 1]
    q = df.iloc[:, 0]
    return scipy.stats.chisquare(p, q).statistic


def Divergence(corpus1, corpus2, difference="KL"):
    """Difference parameter can equal KL, Chi2, or Wass"""
    freqP = nltk.FreqDist(corpus1)
    P = pandas.DataFrame(
        list(freqP.values()), columns=["frequency"], index=list(freqP.keys())
    )
    freqQ = nltk.FreqDist(corpus2)
    Q = pandas.DataFrame(
        list(freqQ.values()), columns=["frequency"], index=list(freqQ.keys())
    )
    if difference == "KL":
        return kl_divergence(P, Q)
    elif difference == "Chi2":
        return chi2_divergence(P, Q)
    elif difference == "KS":
        try:
            return scipy.stats.ks_2samp(P["frequency"], Q["frequency"]).statistic
        except:
            return scipy.stats.ks_2samp(P["frequency"], Q["frequency"])
    elif difference == "Wasserstein":
        try:
            return scipy.stats.wasserstein_distance(
                P["frequency"], Q["frequency"], u_weights=None, v_weights=None
            ).statistic
        except:
            return scipy.stats.wasserstein_distance(
                P["frequency"], Q["frequency"], u_weights=None, v_weights=None
            )

Now we will calculate the KL divergence for each pair of corpora, turn the results into a matrix, and visualize the matrix as a heatmap. Recall that $D_{\mathrm{KL}}(P\|Q)$ measures the amount of information loss when $Q$ is used to approximate $P$. Here, the rows are the $P$s used for calculating KL divergences, and the columns are the $Q$s. So, each cell measures the amount of information loss when the word distribution of the column text is used to approximate the word distribution of the row text. Because the KL divergence is directional, such that the divergence of $P$ from $Q$ is different from the same of $Q$ from $P$, the matrix is assymetric and contains unique information above and below the diagonal. The same is true for the $\chi^2$ divergence.

In [ ]:
len(corpora)

In [ ]:
fileids = list(movie_df["Movie Name"])[0:11]

In [ ]:
super_l = []
for p in corpora:
    sub_l = []
    for q in corpora:
        sub_l.append(Divergence(p, q, difference="KL"))
    super_l.append(sub_l)
M = np.array(super_l)
fig = plt.figure()
div = pandas.DataFrame(M, columns=fileids, index=fileids)
ax = sns.heatmap(div)
plt.show()


To reveal more patterns, let's do a multidimensional scaling of the matrix.

In [ ]:
mds = sklearn.manifold.MDS()
pos = mds.fit(M).embedding_
x = pos[:, 0]
y = pos[:, 1]
fig, ax = plt.subplots(figsize=(6, 6))
plt.plot(x, y, " ")
for i, txt in enumerate(fileids):
    ax.annotate(txt, (x[i], y[i]))

A co-respondent's course and The Chain Gang are away from the others - any guesses why?

We may just want to focus on the distrbution of stopwords or non-stopwords. Let's do the analysis again first for stopwords and then for non-stopwords.

In [ ]:
super_l = []
for p in corpora_s:
    sub_l = []
    for q in corpora_s:
        sub_l.append(Divergence(p, q, difference="KL"))
    super_l.append(sub_l)
M = np.array(super_l)
fig = plt.figure()
div = pandas.DataFrame(M, columns=fileids, index=fileids)
ax = sns.heatmap(div)
plt.show()

In [ ]:
super_l = []
for p in corpora_nons:
    sub_l = []
    for q in corpora_nons:
        sub_l.append(Divergence(p, q, difference="KL"))
    super_l.append(sub_l)
M = np.array(super_l)
fig = plt.figure()
div = pandas.DataFrame(M, columns=fileids, index=fileids)
ax = sns.heatmap(div)
plt.show()

For the KS distance:

In [ ]:
super_l = []
for p in corpora_nons:
    sub_l = []
    for q in corpora_nons:
        sub_l.append(Divergence(p, q, difference="KS"))
    super_l.append(sub_l)
M = np.array(super_l)
fig = plt.figure()
div = pandas.DataFrame(M, columns=fileids, index=fileids)
ax = sns.heatmap(div)
plt.show()

And finally for the Wasserstein or "earth mover's" Distance.

In [ ]:
super_l = []
for p in corpora_nons:
    sub_l = []
    for q in corpora_nons:
        sub_l.append(Divergence(p, q, difference="Wasserstein"))
    super_l.append(sub_l)
M = np.array(super_l)
fig = plt.figure()
div = pandas.DataFrame(M, columns=fileids, index=fileids)
ax = sns.heatmap(div)
plt.show()

You can see that decade X is highly divergent and this makes reading the plot somewhat difficult. Let's fix this by taking the log of each cell, which will reduce the distance of decode X from other texts as an outlier.

In [ ]:
ax = sns.heatmap(div.apply(np.log).replace([np.inf, -np.inf], np.nan))
plt.show()

We only tried 11 movies, and from the same time period, so our signals might not be too interesting. What about different time periods?

An interesting experiment to maybe get some clearer signals: documents by decade in the Davies Corpus Of Historial American English (COHA) and see if we can detect patterns between them.

If we want to rerun this on a new data set of our own composition, we can be a bit more efficient with our coding. Let's use the Shakespeare texts from last week as example. Here we again make use of our convenient `lucem_illud` functions, namely `loadTextDirectory`, which is found in [loaders.py](https://github.com/UChicago-Computational-Content-Analysis/lucem_illud/blob/main/lucem_illud/loaders.py).

In [ ]:
def loadTextDirectory(targetDir, encoding="utf-8"):
    text = []
    fileName = []

    for file in (
        file
        for file in os.scandir(targetDir)
        if file.is_file() and not file.name.startswith(".")
    ):
        with open(file.path, encoding=encoding) as f:
            text.append(f.read())
        fileName.append(file.name)
    return pandas.DataFrame({"text": text}, index=fileName)

In [ ]:
shakespeare_df = lucem_illud.loadTextDirectory("../data/Shakespeare")

Now we stem and remove stopwords

In [ ]:
shakespeare_df["tokenized_text"] = shakespeare_df["text"].apply(
    lambda x: word_tokenize(x)
)
shakespeare_df["normalized_tokens"] = shakespeare_df["tokenized_text"].apply(
    lambda x: normalize_tokens(x)
)

Create the corpus file and generate the heatmap:

In [ ]:
measure = "KS"  # "KL", "Chi2", "KS", "Wasserstein"
num_of_texts = 10  # The bigger this number the slower it will run, you can also try selecting your own plays
fileids_sp = list(shakespeare_df[:num_of_texts].index)
corpora_sp = list(shakespeare_df[:num_of_texts]["normalized_tokens"])
super_l = []
for p in corpora_sp:
    sub_l = []
    for q in corpora_sp:
        sub_l.append(Divergence(p, q, difference=measure))
    super_l.append(sub_l)
M = np.array(super_l)
fig = plt.figure()
div = pandas.DataFrame(M, columns=fileids_sp, index=fileids_sp)
ax = sns.heatmap(div)
plt.show()

What are "The Passionate Pilgrim" and "The Phoenix and the Turtle"? Little known 
poems by Shakespeare that are unsurprisingly hard to classify, as they are so 
different from everything else he wrote.

## <font color="red">*Exercise 7*</font>

<font color="red">Construct cells immediately below this that calculate the KL 
and $\chi^2$ divergences, and the KS and Wasserstein distances between four 
separate corpora, plot these with heatmaps, and then array them in two dimensions 
with multidimensional scaling as above. What does this reveal about relations 
between the corpora? Which analysis (and divergence or distribution) distinguishes 
the authors or documents better?

**Stretch**: Calculate the [Jensen-Shannon Divergence](https://en.wikipedia.org/wiki/Jensen–Shannon_divergence) between your four corpora. What is 
the relationship between the KL and JS divergences?</font>